In [2]:
# @title 00. Importar Pacotes Python
import pandas as pd

from IPython.display import display, Markdown
pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:,.0f}'.format
pd.set_option('display.max_colwidth', None) # mostrar nome completo


In [7]:
# @title 00. Carregar Github
!git clone https://github.com/CAIOZANETTI/estimar_custo


fatal: destination path 'estimar_custo' already exists and is not an empty directory.


In [8]:
import estimar_custo.funcoes as fx

In [ ]:
#utilizar esse dic para armazenar os df
# natural, normalizado, classificado, transformado


dfs = {
    'natural':{'grupos':0,'plan':0,'cpu':0,'ins':0},
    'normalizado':{'grupos':0,'plan':0,'cpu':0,'ins':0},
    'plan_cpu_ins':{'natural':0,'normalizado':0,'apelidado':0},

}
# apelidade, reconhecido, desconhecido...
dfs

{'natural': {'grupos': 0, 'plan': 0, 'cpu': 0, 'ins': 0},
 'normalizado': {'grupos': 0, 'plan': 0, 'cpu': 0, 'ins': 0},
 'plan_cpu_ins': {'natural': 0, 'normalizado': 0}}

## Carregar

### planilha base grupos e apelidos

In [ ]:
df= pd.read_excel('grupos_por_nome.xlsx',sheet_name='agrupamento')
df.fillna(0, inplace=True)
df.head(5)
dfs['natural']['grupos'] = df

### planilha

In [ ]:
def plan_ler_pandas(diretorio,nome):

    caminho = diretorio + nome + ".xlsx"
    df=pd.read_excel(caminho, sheet_name='ler_pandas')
    df['obra'] = nome
    return df


diretorios =['plan_cliente//01_pandas//','plan_gel//01_pandas//']
diretorio_ativo = diretorios[1]

nomes = ["bamin_mina_adutora","gel_ind_copasul_navirai-ms_fev_2024","unipar_civil",'gel_unipar_cubatao_jun_2023','metro_sp_analia_franco']
arquivo_nome = nomes[1]

df = plan_ler_pandas(diretorio_ativo,arquivo_nome)
display(arquivo_nome)
df.rename(columns={'item': 'id'}, inplace=True)
df['origem'] = 'cliente'
display(df.head(3))
dfs['natural']['plan'] = df

'gel_ind_copasul_navirai-ms_fev_2024'

,id,nome,unid,qtd,unit,parcial,arquivo,aba,obra,origem
0,001,COPASUL - PACOTE 5 - ESMAGADORA DE SOJA,0,1,"54,604,451","54,604,451",geral,canteiro,gel_ind_copasul_navirai-ms_fev_2024,cliente
1,001.01,CIVIL,0,0,"54,604,451","54,604,451",geral,canteiro,gel_ind_copasul_navirai-ms_fev_2024,cliente
2,001.01.01,INSTALAÇÕES PROVISÓRIAS,0,0,"3,256,604","3,256,604",geral,canteiro,gel_ind_copasul_navirai-ms_fev_2024,cliente


### cpu e insumos

In [ ]:
diretorio ="G://Meu Drive//00_notebooks//custo//"
arquivo = "custo_direto_14_metro-sp-analia"
caminho = diretorio + arquivo+".xlsx"
dfs['natural']['cpu']=pd.read_excel(caminho, sheet_name='cpu',usecols='A:F',skiprows=1)
dfs['natural']['ins']=pd.read_excel(caminho, sheet_name='ins',usecols='A:J',skiprows=1)

arquivo_split = arquivo.split('_')
obra = arquivo_split[-1]

display('arquivo: '+arquivo)
display('obra: '+obra)

C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


'arquivo: custo_direto_14_metro-sp-analia'

'obra: metro-sp-analia'

### ajustar colunas

#### cpu

In [ ]:
df = dfs['natural']['cpu']

display(Markdown('natural'))
display(df.shape)
display(df.head(3))
df['unit'] = df['mat']+df['mo']
df['parcial'] = df['qtd']*df['unit']

df['origem'] = 'custo'
df['obra'] = obra
df['arquivo'] = arquivo
df['aba'] = 'cpu'

df.drop(['mat', 'mo'], axis=1, inplace=True)
display(Markdown('normalizado'))
display(df.head(3))

dfs['normalizado']['cpu'] = df

natural

(208, 6)

,id,nome,qtd,unid,mat,mo
0,1,"estaca hélice contínua Ø=30cm, em solo SPT < 60",0,m,47,48
1,2,"estaca hélice contínua Ø=40cm, em solo SPT < 60",0,m,83,51
2,3,"estaca hélice contínua Ø=50cm, em solo SPT < 60",0,m,130,64


normalizado

,id,nome,qtd,unid,unit,parcial,origem,obra,arquivo,aba
0,1,"estaca hélice contínua Ø=30cm, em solo SPT < 60",0,m,95,0,custo,metro-sp-analia,custo_direto_14_metro-sp-analia,cpu
1,2,"estaca hélice contínua Ø=40cm, em solo SPT < 60",0,m,134,0,custo,metro-sp-analia,custo_direto_14_metro-sp-analia,cpu
2,3,"estaca hélice contínua Ø=50cm, em solo SPT < 60",0,m,194,0,custo,metro-sp-analia,custo_direto_14_metro-sp-analia,cpu


#### Insumos

In [ ]:
df = dfs['natural']['ins']
display(Markdown('natural'))
display(df.shape)
display(df.head(3))

df['origem'] = 'custo'
df['obra'] = obra
df['arquivo'] = arquivo
df['aba'] = 'ins'

df.drop(['grupo', 'subgrupo','fd'], axis=1, inplace=True)
display(Markdown('Normalizado'))
display(df.head(3))
dfs['normalizado']['ins'] = df

natural

(373, 10)

,id,grupo,subgrupo,tipo,fd,nome,qtd,unid,unit,parcial
0,1,civil,operario,mo,NaN,servente,"1,388",h,45,"61,882"
1,2,civil,operario,mo,NaN,pedreiro,"1,124",h,67,"75,369"
2,3,civil,operario,mo,NaN,carpinteiro,484,h,67,"32,439"


Normalizado

,id,tipo,nome,qtd,unid,unit,parcial,origem,obra,arquivo,aba
0,1,mo,servente,"1,388",h,45,"61,882",custo,metro-sp-analia,custo_direto_14_metro-sp-analia,ins
1,2,mo,pedreiro,"1,124",h,67,"75,369",custo,metro-sp-analia,custo_direto_14_metro-sp-analia,ins
2,3,mo,carpinteiro,484,h,67,"32,439",custo,metro-sp-analia,custo_direto_14_metro-sp-analia,ins


## Concatenar plan+ins+cpu

In [ ]:
df = pd.concat([
    dfs['natural']['plan'],
    dfs['normalizado']['cpu'],
    dfs['normalizado']['ins']
])

display(df.shape)
display(df.head(3))
dfs['plan_cpu_ins']['natural']=df

(985, 11)

,id,nome,unid,qtd,unit,parcial,arquivo,aba,obra,origem,tipo
0,001,COPASUL - PACOTE 5 - ESMAGADORA DE SOJA,0,1,"54,604,451","54,604,451",geral,canteiro,gel_ind_copasul_navirai-ms_fev_2024,cliente,NaN
1,001.01,CIVIL,0,0,"54,604,451","54,604,451",geral,canteiro,gel_ind_copasul_navirai-ms_fev_2024,cliente,NaN
2,001.01.01,INSTALAÇÕES PROVISÓRIAS,0,0,"3,256,604","3,256,604",geral,canteiro,gel_ind_copasul_navirai-ms_fev_2024,cliente,NaN


 ## Normalizar

### Unidades

In [ ]:
df = dfs['plan_cpu_ins']['natural']

df = fx.converter_unidades(df)
display(Markdown('Normalizado'))
display(" ".join(list(df['unid'].unique())))
dfs['plan_cpu_ins']['normalizado'] = df

Normalizado

' vb m3 un m m2 t kg dm3 m3xkm nan h l dia'

### Apelidar e agrupar

In [ ]:
df = dfs['plan_cpu_ins']['normalizado']
df1 = dfs['natural']['grupos']
#display(df1.shape)
df = fx.nome_minusculo(df)
df2 = fx.agrupar_df(df,df1)

pd.set_option('display.max_colwidth', 20) # mostrar nome completo
display(df1.shape)
display(df1.head(3))
pd.set_option('display.max_colwidth', None)

(985, 16)

,id,nome,unid,qtd,unit,parcial,arquivo,aba,obra,origem,tipo,disciplina,etapa,grupo,subgrupo,apelido
0,001,copasul - pacote...,,1,"54,604,451","54,604,451",geral,canteiro,gel_ind_copasul_...,cliente,vazio,vazio,vazio,vazio,vazio,NaN
1,001.01,civil,,0,"54,604,451","54,604,451",geral,canteiro,gel_ind_copasul_...,cliente,vazio,vazio,vazio,vazio,vazio,NaN
2,001.01.01,instalações prov...,,0,"3,256,604","3,256,604",geral,canteiro,gel_ind_copasul_...,cliente,vazio,vazio,vazio,vazio,vazio,NaN


## Desconhecidos

In [ ]:
df_nan = df1[df1['qtd']>0]
df_nan = df_nan[df_nan['apelido'].isna()]

display(df_nan[['id','unid','nome','origem','aba']].head(5))

pivot= df_nan.pivot_table(index=['origem','aba'], values=['nome'],aggfunc='count',margins=True)
display(Markdown(3*"--"))
display(Markdown("**Desconhecidos**: "+str(df_nan.shape[0])))
display(pivot.transpose())

,id,unid,nome,origem,aba
0,001,,copasul - pacote 5 - esmagadora de soja,cliente,canteiro
6,001.01.01.01.01.02,vb,"plano de gestão integrada de sms, sesmt, ambulatório, pgr, pgrs",cliente,canteiro
8,001.01.01.01.01.04,vb,supervisão de obra,cliente,canteiro
11,001.01.01.01.02.01,vb,"estrturas em geral (alvenaria, concreto estrtural, madeira, cercas, postes, hidráulica, el",cliente,canteiro
12,001.01.01.01.02.02,vb,remoção e destinação de entulho,cliente,canteiro


------

**Desconhecidos**: 113

origem  cliente                                     custo      All
aba    canteiro drenagem pavimentação terraplenagem   cpu ins     
nome         58        7           14             6     3  25  113

### Insumos

In [ ]:
df_ins_desconecidos = df_nan[(df_nan['origem']=='custo') & (df_nan['aba']=='ins')]
display(df_ins_desconecidos[['nome','unid']])

,nome,unid
1,pedreiro,h
2,carpinteiro,h
3,armador,h
17,aço para estrutura metalica,kg
21,arame recozido,kg
23,bloco concreto vedação 14x19x39cm,un
24,"bloco concreto estrutural 19x19x39cm, fck=4,5mpa",un
31,brita,m3
35,tinta acrilica,l
45,argamassa polimetrica,kg


### CPU

In [ ]:
df_cpu_desconecidos = df_nan[(df_nan['origem']=='custo') & (df_nan['aba']=='cpu')]
display(df_cpu_desconecidos[['nome','unid']])

,nome,unid
140,alvenaria bloco concreto cobogó 08x50x40cm,m2
152,revestimento ceramico extrudado gail,m2
156,revestimento piso cimento queimado,m2


## Conhecidos

### Planilha

In [ ]:
df2 = df1[df1['origem']=='cliente']

pivot_unid_apelido_nome= df2.pivot_table(index=['unid','apelido','nome',],
                              values=['qtd'],#values=['unit','qtd', 'parcial'],
                              aggfunc='sum')

pd.set_option('display.max_rows', None)
display(pivot_unid_apelido_nome)

qtd
unid apelido                            nome                                                                                              
kg   armadura_passiva_vergalhão         aço ca-50 fornecimento, dobra, colocação                                                     1,205
                                        aço ca-60 fornecimento, dobra, colocação                                                    15,188
     concreto_fibra_geral               fibra de polipropileno                                                                       2,088
m    canaleta-geral                     calha meio tubo 400mm                                                                        2,100
     cerca_cerca_geral                  cerca definitiva em tela com altura de 2,60m  em todo o perimetro do terreno, com mourões    3,250
                                        cerca provisória para isolar o caminho de serviço, com chapa de aluzinco ondulado n°28 100     820
     dreno_pavimento_geral              dreno de pavimento dss-04                                                                   11,210
     junta-serrada                      serradas - js                                                                                3,954
     meio-fio-geral                     meio fio simples                                                                             2,630
     meio-fio-sarjeta                   meio fio com sarjeta                                                                        13,390
     piso_armadura_treliça              treliça h=16cm (apoio tela)                                                                 12,652
                                        treliça h=9cm (apoio barras transferência)                                                  10,596
     tubo-concreto_tubo_geral           tubulação concreto ou pead corrugado dn=750mm                                                  100
                                        tubulação concreto ou pead corrugado dn=900mm                                                  180
     tubo-concreto_tubo_Ø100cm          tubulação concreto ou pead corrugado dn=1000mm                                                 576
     tubo-concreto_tubo_Ø120cm          tubulação concreto ou pead corrugado dn=1200mm                                               1,140
     tubo-concreto_tubo_Ø150cm          tubulação concreto ou pead corrugado dn=1500mm                                                 380
     tubo-concreto_tubo_Ø40cm           tubulação concreto ou pead corrugado dn=400mm (pead considerar dn 375mm)                     3,627
     tubo-concreto_tubo_Ø60cm           tubulação concreto ou pead corrugado dn=600mm                                                2,247
     tubo-concreto_tubo_Ø80cm           tubulação concreto ou pead corrugado dn=800mm                                                  402
m2   alvenaria_revestimento_reboco      execução de paredes em alvenaria (incluindo estrutura, chapisco e reboco)                      115
     alvenaria_vedação_tijolo           sumidouro em alvenaria dobrada de tijolo cerâmico furado                                        30
     armadura_passiva_vergalhão         tela de reforço q-159 - armadura superior                                                   18,187
     cobertura_telha-aço_geral          telha de aluzinco                                                                              635
     compactação-regularização          assentamento/compactação                                                                    40,500
                                        regularização do sub-leito - a compactação a 100% pi já incluso na terraplenagem            15,815
                                        regularização do subleito                                                                  168,883
                                        regularização e selecionamento do revestimento primário aplicado

### Cpu

In [ ]:


df2 = df1[(df1['origem']=='custo') & (df1['aba']=='cpu')]

pivot_unid_apelido_nome= df2.pivot_table(index=['unid','apelido','nome',],
                              values=['qtd'],#values=['unit','qtd', 'parcial'],
                              aggfunc='sum')

pd.set_option('display.max_rows', None)
display(df2.shape)
display(pivot_unid_apelido_nome)

(208, 16)

qtd
unid  apelido                                nome                                                                          
dm3   estrutura_grout_geral                  graute                                                                       0
kg    armadura_passiva_vergalhão             armadura aço ca-50 ca-60                                                     0
                                             armadura aço ca-50 ca-60 para estaca helice                                  0
m     cabo_aterramento_geral                 aterramento elétrico de aço ca-25 ø=16mm                                     0
      cabo_elétrico_geral                    aterramento elétrico com cabo de aço zincado, ø=3/8", fios=7                 0
      canaleta-geral                         canaleta de drenagem ø40cm                                                   0
      cerca_cerca_concreto                   cerca em mourão concreto                                                     0
      demolição_concreto_furo                furo em estutura de concreto ø12,5mm                                         0
      dreno_pavimento_geral                  dreno pavimento ø100mm                                                       0
      estaca_helice_geral                    estaca hélice contínua ø=30cm, em solo spt < 60                              0
                                             estaca hélice contínua ø=40cm, em solo spt < 60                              0
                                             estaca hélice contínua ø=50cm, em solo spt < 60                              0
                                             estaca hélice contínua ø=60cm, em solo spt < 60                              0
      estaca_pre-moldado_Ø25cm               estaca concreto pre-moldada estrela (mat+mo) ø=27cm                          0
      estaca_pre-moldado_Ø35cm               estaca concreto pre-moldada estrela (mat+mo) ø=36cm                          0
      estaca_pre-moldado_Ø40cm               estaca concreto pre-moldada estrela (mat+mo) ø=40cm                          0
      estaca_pre-moldado_Ø45cm               estaca concreto pre-moldada estrela (mat+mo) ø=45cm                          0
      estaca_pre-moldado_Ø50cm               estaca concreto pre-moldada estrela (mat+mo) ø=52cm                          0
      estaca_raiz_solo                       estaca raiz em solo (mat+mo) ø=30cm                                          0
                                             estaca raiz em solo (mat+mo) ø=40cm                                          0
                                             estaca raiz em solo (mat+mo) ø=45cm                                          0
                                             estaca raiz em solo (mat+mo) ø=50cm                                          0
      infra_eletroduto_geral                 eletroduto flexível ø1"                                                      0
      meio-fio-extrusado                     meio foi concreto extrudado                                                  0
      meio-fio-pre-moldado                   meio fio concreto pré-moldado                                              122
      meio-fio-sarjeta                       meio fio pré-moldado com sarjeta                                             0
      prfv_tubo_geral                        motagem tubo prfv                                                            0
      revestimento_rodape_ceramico-extrudado rodape ceramico extrudado gail                                              19
      revestimento_rodape_concreto           rodapé concreto pre-moldado                                                225
      sarjeta-concreto                       sarjeta em concreto extrudado                                                0
                                             sarjeta em concreto moldada in-loco                                          0
      tubo-aço_montagem     

## Salvar

In [ ]:
salvar = 'plan_cliente//03_relatorio//'+arquivo_nome+'_plan_agrupada.xlsx'

#display(df1)
with pd.ExcelWriter(salvar) as writer:
    # Save each DataFrame to a specific sheet
    df1.to_excel(writer, sheet_name='00_plan', index=False)
    pivot_unid_apelido_nome.to_excel(writer, sheet_name='01_unid_apelido_nome', index=True,merge_cells=False)

display(salvar)

'plan_cliente//03_relatorio//gel_ind_copasul_navirai-ms_fev_2024_plan_agrupada.xlsx'

## Analise

### Apelidos Usados

In [ ]:
#filtro somente da planilha!!!
df2 = df1[df1['origem']=='cliente']

apelidos_usados_plan=df2['apelido'].unique()
display('apelidos encontrados plan cliente: '+ str(len(apelidos_usados_plan)))

'apelidos encontrados plan cliente: 89'

In [ ]:
#filtro somente da planilha!!!
df2 = df1[df1['origem']=='custo']

apelidos_usados_cpu=df2['apelido'].unique()
display('apelidos encontrados plan custo: '+ str(len(apelidos_usados_cpu)))

'apelidos encontrados plan custo: 135'

### CPU

In [ ]:
df2 = df1[(df1['origem']=='custo') & (df1['aba']=='cpu')]
apelidos_cpu=df2['apelido'].unique()
display(apelidos_cpu)
pivot_unid_apelido_nome= df2.pivot_table(index=['unid','apelido','nome',],
                              values=['qtd'],#values=['unit','qtd', 'parcial'],
                              aggfunc='sum')
display(pivot_unid_apelido_nome)

array(['estaca_helice_geral', 'concreto_usinado_>41Mpa',
       'concreto_usinado_20-30Mpa', 'concreto_usinado_30-40Mpa',
       'concreto_estrutural_geral', 'escavação_vala-manual_geral',
       'escavação_vala-mecanica_1a', 'demolição_estaca_arrasamento', nan,
       'armadura_passiva_vergalhão', 'concreto_piso_geral',
       'revestimento_piso_ceramica', 'forma_metalica_geral', 'paver_10',
       'pintura_acrilica_geral', 'compactação_vala_compactação',
       'cabo_aterramento_geral', 'cabo_elétrico_geral',
       'infra_eletroduto_geral', 'compactação_ceu-aberto_solo',
       'alvenaria_estrutural_bloco', 'alvenaria_revestimento_chapisco',
       'alvenaria_revestimento_emboço', 'alvenaria_revestimento_reboco',
       'tubo-pead_geral', 'tubo-concreto_bstc_geral',
       'meio-fio-pre-moldado', 'sarjeta-concreto',
       'pavimento_sub-base_sub-base', 'demolição',
       'forma_cimbramento_geral', 'tapume', 'cobertura_telha-aço_geral',
       'compactação-regularização', 'forro_pv

qtd
unid  apelido                                nome                                                                          
dm3   estrutura_grout_geral                  graute                                                                       0
kg    armadura_passiva_vergalhão             armadura aço ca-50 ca-60                                                     0
                                             armadura aço ca-50 ca-60 para estaca helice                                  0
m     cabo_aterramento_geral                 aterramento elétrico de aço ca-25 ø=16mm                                     0
      cabo_elétrico_geral                    aterramento elétrico com cabo de aço zincado, ø=3/8", fios=7                 0
      canaleta-geral                         canaleta de drenagem ø40cm                                                   0
      cerca_cerca_concreto                   cerca em mourão concreto                                                     0
      demolição_concreto_furo                furo em estutura de concreto ø12,5mm                                         0
      dreno_pavimento_geral                  dreno pavimento ø100mm                                                       0
      estaca_helice_geral                    estaca hélice contínua ø=30cm, em solo spt < 60                              0
                                             estaca hélice contínua ø=40cm, em solo spt < 60                              0
                                             estaca hélice contínua ø=50cm, em solo spt < 60                              0
                                             estaca hélice contínua ø=60cm, em solo spt < 60                              0
      estaca_pre-moldado_Ø25cm               estaca concreto pre-moldada estrela (mat+mo) ø=27cm                          0
      estaca_pre-moldado_Ø35cm               estaca concreto pre-moldada estrela (mat+mo) ø=36cm                          0
      estaca_pre-moldado_Ø40cm               estaca concreto pre-moldada estrela (mat+mo) ø=40cm                          0
      estaca_pre-moldado_Ø45cm               estaca concreto pre-moldada estrela (mat+mo) ø=45cm                          0
      estaca_pre-moldado_Ø50cm               estaca concreto pre-moldada estrela (mat+mo) ø=52cm                          0
      estaca_raiz_solo                       estaca raiz em solo (mat+mo) ø=30cm                                          0
                                             estaca raiz em solo (mat+mo) ø=40cm                                          0
                                             estaca raiz em solo (mat+mo) ø=45cm                                          0
                                             estaca raiz em solo (mat+mo) ø=50cm                                          0
      infra_eletroduto_geral                 eletroduto flexível ø1"                                                      0
      meio-fio-extrusado                     meio foi concreto extrudado                                                  0
      meio-fio-pre-moldado                   meio fio concreto pré-moldado                                              122
      meio-fio-sarjeta                       meio fio pré-moldado com sarjeta                                             0
      prfv_tubo_geral                        motagem tubo prfv                                                            0
      revestimento_rodape_ceramico-extrudado rodape ceramico extrudado gail                                              19
      revestimento_rodape_concreto           rodapé concreto pre-moldado                                                225
      sarjeta-concreto                       sarjeta em concreto extrudado                                                0
                                             sarjeta em concreto moldada in-loco                                          0
      tubo-aço_montagem     

### Custo

In [ ]:
apelidos_usados_plan

df1 =df1[df1['apelido'].isin(apelidos_usados_plan)]
df2 =df1[(df1['apelido'].notnull())&(df1['aba']!='ins')]
pivot2= df2.pivot_table(index=['unid','apelido','origem','nome','unit'],
                              values=['qtd'],#values=['unit','qtd', 'parcial'],
                              aggfunc='sum')
display(pivot2)

qtd
unid apelido                            origem  nome                                                                                       unit         
kg   armadura_passiva_vergalhão         cliente aço ca-50 fornecimento, dobra, colocação                                                   0       1,205
                                                aço ca-60 fornecimento, dobra, colocação                                                   0      15,188
                                        custo   armadura aço ca-50 ca-60                                                                   11          0
                                                armadura aço ca-50 ca-60 para estaca helice                                                11          0
     concreto_fibra_geral               cliente fibra de polipropileno                                                                     45      2,088
m    canaleta-geral                     cliente calha meio tubo 400mm                                                                      63      2,100
                                        custo   canaleta de drenagem ø40cm                                                                 79          0
     cerca_cerca_geral                  cliente cerca definitiva em tela com altura de 2,60m  em todo o perimetro do terreno, com mourões  401     3,250
                                                cerca provisória para isolar o caminho de serviço, com chapa de aluzinco ondulado n°28 100 427       820
     dreno_pavimento_geral              cliente dreno de pavimento dss-04                                                                  73     11,210
                                        custo   dreno pavimento ø100mm                                                                     44          0
     junta-serrada                      cliente serradas - js                                                                              31      3,954
     meio-fio-geral                     cliente meio fio simples                                                                           63      2,630
     meio-fio-sarjeta                   cliente meio fio com sarjeta                                                                       110    13,390
                                        custo   meio fio pré-moldado com sarjeta                                                           83          0
     piso_armadura_treliça              cliente treliça h=16cm (apoio tela)                                                                8      12,652
                                                treliça h=9cm (apoio barras transferência)                                                 6      10,596
     tubo-concreto_tubo_geral           cliente tubulação concreto ou pead corrugado dn=750mm                                              685       100
                                                tubulação concreto ou pead corrugado dn=900mm                                              817       180
     tubo-concreto_tubo_Ø100cm          cliente tubulação concreto ou pead corrugado dn=1000mm                                             1,286     576
     tubo-concreto_tubo_Ø120cm          cliente tubulação concreto ou pead corrugado dn=1200mm                                             1,368   1,140
     tubo-concreto_tubo_Ø150cm          cliente tubulação concreto ou pead corrugado dn=1500mm                                             1,844     380
     tubo-concreto_tubo_Ø40cm           cliente tubulação concreto ou pead corrugado dn=400mm (pead considerar dn 375mm)                   233     3,627
     tubo-concreto_tubo_Ø60cm           cliente tubulação concreto ou pead corrugado dn=600mm                                              498     2,247
     tubo-concreto_tubo_Ø80cm           cliente tubulação concreto ou pead corrugado dn=800mm                                              895       402
m2   alvenaria_rev